### Layers : Fundamental blocks of Neural Network

In [1]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable



### Linear(in_features,out_features,bias)

定義 輸入和輸出層的維度(weight的維度)

In [2]:
myLayer = Linear(in_features=10,out_features=5,bias=True)
inp = Variable(torch.randn(1,10))
myLayer = Linear(in_features=10,out_features=5,bias=True) 
myLayer(inp)

tensor([[ 0.6138, -0.6005,  0.1217,  1.1433,  1.2532]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[-0.2563, -0.2979,  0.2679, -0.0872, -0.2517,  0.2666, -0.2537,  0.2673,
         -0.1313,  0.0095],
        [ 0.2295,  0.1175,  0.0512, -0.1537, -0.2360, -0.2800, -0.0673, -0.0395,
         -0.0121,  0.2237],
        [ 0.0227,  0.1262, -0.1672, -0.2167, -0.0580,  0.2146,  0.2710, -0.1066,
         -0.2871, -0.0474],
        [-0.3156,  0.0862, -0.1411,  0.0072, -0.1518, -0.1100, -0.1887,  0.0707,
         -0.0701, -0.1305],
        [-0.2692,  0.1937, -0.0219, -0.0184, -0.1888,  0.3106,  0.1528, -0.2884,
         -0.2803,  0.2189]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([ 0.0489, -0.1796,  0.0931,  0.2196,  0.0039], requires_grad=True)

### Stacking Linear layers

建立多層神經層透過迭代

In [5]:
myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

tensor([[0.2094, 0.1383]], grad_fn=<AddmmBackward>)

### PyTorch Non-linear Activations

使用兩種呼叫激勵函數的不同方式

In [6]:
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
myRelu = ReLU()
myRelu(sample_data)


tensor([[1., 2., 0., 0.]])

In [7]:
import torch.nn as nn
import torch.nn.functional as F
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
f = F.relu(sample_data) # Much simpler.
f

tensor([[1., 2., 0., 0.]])

### Neural Network 

利用類別建立多層神經網路

In [8]:
class MyFirstNetwork(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyFirstNetwork,self).__init__() 
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def layer(self):
        first  = [self.layer1.weight.data,self.layer1.bias.data]
        second = [self.layer2.weight.data,self.layer2.bias.data]
        return first,second
    def forward(self,input): 
        out = self.layer1(input) 
        out = F.relu(out)
        out = self.layer2(out) 
        out = F.softmax(out,dim=1)
        return out

model = MyFirstNetwork(224*224,56*56,2)
print(model)

MyFirstNetwork(
  (layer1): Linear(in_features=50176, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)


### Loss

介紹兩種不同的loss function

In [9]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

分類問題常用的loss function: 交叉熵(cross-entropy)

In [10]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)

### loss(x,class)=−x[class]+log(∑exp(x[j]))

利用上述公式自訂函數與內建函數比較

In [11]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
print(output.data.item())

def CrossEntropyLoss(input,target):
    sum = 0
    for i in range(len(input)):
        tmp = 0
        for j in range(len(input[0])):
            tmp+=np.e**(input[i][j])
        sum+=-input[i][target[i]]+np.log(tmp)
    return float(sum)/len(input)

print(CrossEntropyLoss(input.data.numpy(),target.data.numpy()))

1.7645517587661743
1.764551785993138


### Optimizer

將多個圖形利用自訂的類別建立成一個物件

In [12]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
# ### Optimizer
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(224,224)):
        self.files = glob(root_dir)
        self.size = size
        self.arr = np.array([[1,0],[0,1]],float)
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        input = Variable(torch.from_numpy(np.asarray(Image.open(self.files[idx]).resize((1,224*224)), float)))
        if(idx<1000):
            target = Variable(torch.from_numpy(self.arr[0]))
        else:
            target = Variable(torch.from_numpy(self.arr[1])) 
        return input,target

dataset=DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/classifier/*.jpg")

### Training

In [13]:
import torch.optim as optim
from time import perf_counter
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001,momentum=0.9)
print("Training......")

index = 0
t1 = perf_counter()
t  = t1
for input, target in dataset:
    data=input[:,:,0].view(1,-1).float()+input[:,:,1].view(1,-1).float()+input[:,:,2].view(1,-1).float()
    output = model(data/(3*(255.)**2)**0.5)
    optimizer.zero_grad()
    loss = loss_fn(output, target.float())
    loss.backward()
    optimizer.step()
    index+=1
    if(perf_counter()-t > 30):
        t = perf_counter()
        print("Completion ratio :",str(index*100.0/2000)+"%")
t2  = perf_counter()

print(model)
print("--------------------------------------")
print("layer1 :","\nweight :\n",model.layer()[0][0].numpy(),"\nbias :\n",model.layer()[0][1].numpy())
print("")
print("layer2 :","\nweight :\n",model.layer()[1][0].numpy(),"\nbias :\n",model.layer()[1][1].numpy())
print("time :",str((t2-t1)/60.0)+" min")

print("\ntest.....")
test = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/sample_test/*.jpg")
data1=test[0][0][:,:,0].view(1,-1).float()+test[0][0][:,:,1].view(1,-1).float()+test[0][0][:,:,2].view(1,-1).float()
data2=test[1][0][:,:,0].view(1,-1).float()+test[1][0][:,:,1].view(1,-1).float()+test[1][0][:,:,2].view(1,-1).float()
output1 = model(data1/(3*(255.)**2)**0.5)
output2 = model(data2/(3*(255.)**2)**0.5)
print(output1)
print(output2)

Training......
Completion ratio : 2.4%
Completion ratio : 4.8%
Completion ratio : 7.15%
Completion ratio : 9.5%
Completion ratio : 11.85%
Completion ratio : 14.15%
Completion ratio : 16.35%
Completion ratio : 18.65%
Completion ratio : 21.0%
Completion ratio : 23.35%
Completion ratio : 25.7%
Completion ratio : 28.05%
Completion ratio : 30.35%
Completion ratio : 32.65%
Completion ratio : 34.9%
Completion ratio : 37.1%
Completion ratio : 39.4%
Completion ratio : 41.7%
Completion ratio : 44.0%
Completion ratio : 46.3%
Completion ratio : 48.6%
Completion ratio : 50.9%
Completion ratio : 53.2%
Completion ratio : 55.5%
Completion ratio : 57.75%
Completion ratio : 60.0%
Completion ratio : 62.25%
Completion ratio : 64.55%
Completion ratio : 66.85%
Completion ratio : 69.15%
Completion ratio : 71.45%
Completion ratio : 73.75%
Completion ratio : 76.0%
Completion ratio : 78.25%
Completion ratio : 80.5%
Completion ratio : 82.75%
Completion ratio : 85.0%
Completion ratio : 87.25%
Completion ratio : 8